# 📊 Nursing FunctionGemma Evaluation

This notebook evaluates the **Nursing FunctionGemma** model's function-calling accuracy.

**Metrics Calculated:**
- Overall Accuracy
- Function Type Accuracy (vitals, medication, NMC search)
- Parameter Extraction Accuracy
- Error Analysis

In [ ]:
!pip install -q -U transformers peft torch accelerate bitsandbytes huggingface_hub pandas matplotlib seaborn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import torch
import json
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from tqdm import tqdm

# Load evaluation dataset
EVAL_PATH = "/content/drive/MyDrive/nmc_brain/data/function_eval_dataset.json"

with open(EVAL_PATH, 'r') as f:
    eval_data = json.load(f)

print(f"Loaded {len(eval_data)} evaluation examples")

In [ ]:
# Load model
BASE_MODEL_ID = "google/medgemma-4b-it"
ADAPTER_ID = "NurseCitizenDeveloper/nursing-function-gemma"

print("Loading base model...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

print(f"Loading adapter from: {ADAPTER_ID}")
model = PeftModel.from_pretrained(model, ADAPTER_ID)
print("✅ Model loaded!")

In [ ]:
def generate_prediction(user_input):
    """Generate a function call prediction."""
    tools_prompt = """You are a clinical AI agent. You have access to the following tools:
- record_vitals(systolic, diastolic, heart_rate, temp_c)
- administer_medication(drug_name, dose, route)
- search_nmc_standards(query)
If the user's request requires a tool, output the function call XML."""

    prompt = f"<start_of_turn>user\n{tools_prompt}\n\n{user_input}<end_of_turn>\n<start_of_turn>model\n"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=False,
            temperature=0.1,
            eos_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    if "<start_of_turn>model" in response:
        response = response.split("<start_of_turn>model")[-1].strip()
    if "<end_of_turn>" in response:
        response = response.split("<end_of_turn>")[0].strip()
    
    # Extract function call
    match = re.search(r'<function_call>(.*?)</function_call>', response, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # Try to find raw function call
    for func_name in ['record_vitals', 'administer_medication', 'search_nmc_standards']:
        if func_name in response:
            match = re.search(rf'{func_name}\(.*?\)', response)
            if match:
                return match.group(0)
    
    return response

In [ ]:
# Run evaluation
print("Running evaluation...")
results = []

for item in tqdm(eval_data):
    user_input = item['input']
    expected = item['expected']
    predicted = generate_prediction(user_input)
    
    # Determine function type
    if 'record_vitals' in expected:
        func_type = 'vitals'
    elif 'administer_medication' in expected:
        func_type = 'medication'
    else:
        func_type = 'nmc_search'
    
    # Check correctness
    # Function name match
    func_name_expected = expected.split('(')[0]
    func_name_correct = func_name_expected in predicted
    
    # Exact match (normalized)
    expected_norm = re.sub(r'\s+', '', expected.lower())
    predicted_norm = re.sub(r'\s+', '', predicted.lower())
    exact_match = expected_norm == predicted_norm
    
    results.append({
        'input': user_input,
        'expected': expected,
        'predicted': predicted,
        'func_type': func_type,
        'func_name_correct': func_name_correct,
        'exact_match': exact_match
    })

df = pd.DataFrame(results)
print(f"\n✅ Evaluation complete! {len(df)} examples processed.")

## 📈 Results Summary

In [ ]:
# Calculate overall metrics
overall_accuracy = df['exact_match'].mean() * 100
func_name_accuracy = df['func_name_correct'].mean() * 100

print("="*50)
print("📊 OVERALL RESULTS")
print("="*50)
print(f"\n🎯 Exact Match Accuracy: {overall_accuracy:.1f}%")
print(f"🔧 Function Name Accuracy: {func_name_accuracy:.1f}%")
print(f"📝 Total Test Cases: {len(df)}")

In [ ]:
# Accuracy by function type
print("\n" + "="*50)
print("📊 ACCURACY BY FUNCTION TYPE")
print("="*50)

for func_type in ['vitals', 'medication', 'nmc_search']:
    subset = df[df['func_type'] == func_type]
    if len(subset) > 0:
        acc = subset['exact_match'].mean() * 100
        func_acc = subset['func_name_correct'].mean() * 100
        print(f"\n{func_type.upper()}:")
        print(f"  - Count: {len(subset)}")
        print(f"  - Exact Match: {acc:.1f}%")
        print(f"  - Function Name Match: {func_acc:.1f}%")

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Pie chart - overall accuracy
correct = df['exact_match'].sum()
incorrect = len(df) - correct
axes[0].pie([correct, incorrect], labels=['Correct', 'Incorrect'], 
            autopct='%1.1f%%', colors=['#2ecc71', '#e74c3c'])
axes[0].set_title('Overall Exact Match Accuracy')

# Bar chart - accuracy by function type
func_types = ['vitals', 'medication', 'nmc_search']
accuracies = [df[df['func_type']==ft]['exact_match'].mean()*100 for ft in func_types]
bars = axes[1].bar(func_types, accuracies, color=['#3498db', '#9b59b6', '#e67e22'])
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Accuracy by Function Type')
axes[1].set_ylim(0, 100)
for bar, acc in zip(bars, accuracies):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
                 f'{acc:.1f}%', ha='center')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/nmc_brain/evaluation_results.png', dpi=150)
plt.show()
print("\n📸 Chart saved to: /content/drive/MyDrive/nmc_brain/evaluation_results.png")

## 🔍 Error Analysis

In [ ]:
# Show errors
errors = df[df['exact_match'] == False]

print(f"\n❌ Total Errors: {len(errors)} / {len(df)}")
print("\n" + "="*50)
print("ERROR EXAMPLES")
print("="*50)

for i, row in errors.head(10).iterrows():
    print(f"\n--- Example {i+1} ---")
    print(f"Input: {row['input']}")
    print(f"Expected: {row['expected']}")
    print(f"Predicted: {row['predicted']}")
    print(f"Function Name Correct: {row['func_name_correct']}")

In [ ]:
# Save results to CSV
df.to_csv('/content/drive/MyDrive/nmc_brain/evaluation_results.csv', index=False)
print("\n💾 Results saved to: /content/drive/MyDrive/nmc_brain/evaluation_results.csv")

## 📋 Summary Statistics for Article

In [ ]:
# Generate summary for article
print("\n" + "="*60)
print("📋 SUMMARY STATISTICS FOR ARTICLE")
print("="*60)
print(f"""
The Nursing FunctionGemma model was evaluated on a held-out test set
of {len(df)} clinical scenarios across three function types:

OVERALL PERFORMANCE:
- Exact Match Accuracy: {overall_accuracy:.1f}%
- Function Name Accuracy: {func_name_accuracy:.1f}%

PERFORMANCE BY FUNCTION TYPE:
""")

for func_type in ['vitals', 'medication', 'nmc_search']:
    subset = df[df['func_type'] == func_type]
    if len(subset) > 0:
        acc = subset['exact_match'].mean() * 100
        print(f"- {func_type.title()}: {acc:.1f}% (n={len(subset)})")

print(f"""
TRAINING DETAILS:
- Base Model: google/medgemma-4b-it
- Fine-tuning Method: QLoRA (4-bit quantization)
- Training Examples: 500
- Epochs: 5
- Final Training Loss: 0.74

The model demonstrates strong capability in converting natural language
clinical statements into structured function calls, with particularly
high accuracy in {df.groupby('func_type')['exact_match'].mean().idxmax()} tasks.
""")